In [1]:
import pandas as pd
import numpy as np

In [2]:
# definir arquivo Pickle de entrada
arquivo = 'afastamentos-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf05.csv'

df = pd.read_pickle(arquivo)
df.head()

,SEXO,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,,
MARIA,0,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,1,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
FRANCISCO,1,15,3,19,9,3,14,1,18,6,0,0,0,0,0,0,0
JOAO,1,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
df.shape

(9726, 17)

In [4]:
df.drop(df[df['SEXO'] > 1].index, inplace=True)

In [5]:
df.shape

(8711, 17)

In [6]:
X = df.drop(['SEXO'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo
print('Forma dos dados:', X.shape, y.shape)

Forma dos dados: (8711, 16) (8711,)


In [7]:
# dividir dados em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

print("Shapes:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Shapes: (5836, 16) (2875, 16) (5836,) (2875,)


https://github.com/hyperopt/hyperopt-sklearn

In [9]:
from hpsklearn import HyperoptEstimator
#from hpsklearn import any_classifier, any_preprocessing, any_sparse_classifier
from hyperopt import tpe
import hpsklearn as hs

In [10]:
# Instantiate a HyperoptEstimator with the search space and number of evaluations
automl = HyperoptEstimator(
    classifier=hs.any_sparse_classifier('my_clf'),
    preprocessing=[], #hs.any_preprocessing('my_pre'),
    algo=tpe.suggest,
    max_evals=10,
    trial_timeout=2*60
)
automl

In [14]:
type(automl)

hpsklearn.estimator.hyperopt_estimator

In [11]:
# Search the hyperparameter space based on the data
automl.fit(X_train, y_train)

 83%|████████▎ | 5/6 [00:00<?, ?trial/s, best loss=?]

/opt/anaconda3/lib/python3.6/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)



100%|██████████| 10/10 [00:00<00:00,  3.36trial/s, best loss: 0.04195205479452058]


/opt/anaconda3/lib/python3.6/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [12]:
# Show the results
print(automl.score(X_test, y_test))

0.9537391304347826


In [13]:
print(automl.best_model())

{'learner': LinearSVC(C=0.02763349905571472, class_weight='balanced', dual=False,
          intercept_scaling=3.4599306440881423, multi_class='crammer_singer',
          random_state=1, tol=0.0017150716231446451, verbose=False), 'preprocs': (), 'ex_preprocs': ()}


In [58]:
# carregar dados de teste

In [21]:
# definir arquivo Pickle de entrada
#arquivo = 'afastamentos-nomes-sexo-preparado.pkl.bz2'
arquivo = 'cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf05.csv'

df2 = pd.read_pickle(arquivo)
df2.head()

,SEXO,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,,
MARIA,0,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANA,0,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,1,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,1,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0


In [22]:
df2.shape

(48970, 17)

In [27]:
df2.drop(df2[df2['SEXO'] > 1].index, inplace=True)

In [28]:
df2.shape

(41150, 17)

In [29]:
X_test = df2.drop(['SEXO'], axis=1) # tudo, exceto a coluna alvo
y_test = df2['SEXO'] # apenas a coluna alvo

In [30]:
y_pred = automl.predict(X_test)
y_pred

array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)

In [31]:
from sklearn.metrics import accuracy_score

print("Accuracy score: %.2f %%" % (accuracy_score(y_test, y_pred) * 100))

Accuracy score: 95.26 %


In [32]:
from sklearn.metrics import confusion_matrix

print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

Confusion matrix:
 [[20530  1361]
 [  591 18668]]
